<h1>ライブラリのimportと環境確認</h1>


In [16]:
import torch, torchvision
print(torch.__version__)
print(torchvision.__version__)


2.5.1
0.20.1


In [17]:
#必要なライブラリーのinstall
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import math
from torch.utils.data import Dataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import make_grid
import warnings
warnings.filterwarnings("ignore")

from typing import List, Dict, Tuple, Optional
import copy
import math
from omegaconf import OmegaConf,DictConfig

from functools import partial
from tqdm import tqdm
import torch.nn.init as init

from typing import Dict, List, Union, Callable, Optional
from IPython.display import display
from matplotlib import rcParams
from matplotlib.ticker import FuncFormatter

import itertools
from scipy import stats

In [18]:
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("GPU name:", torch.cuda.get_device_name(0))


CUDA available: True
CUDA version: 12.1
GPU name: NVIDIA GeForce RTX 3060


<h1>データの読み込みと前処理</h1>


In [19]:
#天気のデータを読み込む
def load_weather_data(filepath, skiprows=3, region_name="地域名"):
    """
    天気CSVを読み込み、30分単位の時系列データに変換する関数。
    region_nameの引数は、地域の区別がないファイル使用時に役立つ
    今回は、地域別のファイルのためregion_nameの引数を排除しても良い
    Parameters:
    - filepath (str): CSVファイルのパス
    - skiprows (int): 読み飛ばす行数（データ開始まで）
    - region_name (str): 地域識別用の名前（'identifier'列として追加）
    Returns:
    - DataFrame: 1時間間隔の天気データ
    """
    df = pd.read_csv(filepath, encoding="shift_jis", skiprows=skiprows)
    # CSV 読み込み（欠損は0で埋める）
    df = df.fillna(method="bfill") # 次の値で埋める
    df = df.fillna(method="ffill")  # 前の値で埋める（bfill で埋まらなかったもの）
    df = df.fillna(0)               # それでも埋まらないものは 0 に

    # 不要な最初の行の除去と列名の整理
    df = df.iloc[2:].reset_index(drop=True)
    df = df.loc[:, ~df.columns.str.contains(r"\.\d+$")]  # ".数字"で終わる列を除去

    # 日時を datetime に変換して index に設定
    df["年月日時"] = pd.to_datetime(df["年月日時"])
    df = df.set_index("年月日時")

     # 地域識別列の追加
    df["identifier"] = region_name

    # カラム順を調整（任意）
    cols = ["identifier"] + [col for col in df.columns if col != "identifier"]
    df = df[cols]
    # 数値列だけ抽出して resample
    numeric_cols = df.select_dtypes(include=["number"]).columns
    df_numeric = df[numeric_cols]
    df_hourly = df[numeric_cols]

    # identifier はカテゴリ列なので補完
    df_hourly["identifier"] = region_name

    return df_hourly

In [20]:
def load_and_concat_multiple(files, region_name, skiprows_dict=None):
    """複数の天気ファイルを読み込み、連結する（ファイルごとに skiprows を指定可能）
    　　読み込みファイルが分割されてる時に一度で処理できる
      　load_weather_data関数と同じようにregion_name引数は排除可能
    引数:
    - files: 読み込む天気データファイルのリスト（各ファイルは文字列で指定）
    - region_name: 各データフレームに付加する地域名
    - skiprows_dict: 各ファイルに対応するスキップ行数を指定する辞書（この変数を作る必要がある）
    戻り値:
    - combined: 連結されたデータフレーム
    """
    dfs = []
    for f in files:
        skiprows = skiprows_dict.get(f, 0) if skiprows_dict else 0
        df = load_weather_data(f, skiprows=skiprows, region_name=region_name)
        dfs.append(df)
    
    combined = pd.concat(dfs).sort_index()
    combined = combined[~combined.index.duplicated()]
    return combined

In [23]:
# 各地域（四国、中国）の天気ファイル一覧を指定
# 四国の天気データ
file_list_shikoku = [
    data_dir + "高松天気2022-04to06.csv",  # 2022年4月から6月までのデータ
    data_dir + "高松天気20227to9.csv",     # 2022年7月から9月までのデータ
    data_dir + "高松天気202210to12.csv",  # 2022年10月から12月までのデータ
    data_dir + "高松天気20231to3.csv"     # 2023年1月から3月までのデータ
]

# 中国の天気データ
file_list_chugoku = [
    data_dir + "岡山天気2022-04to06.csv",  # 2022年4月から6月までのデータ
    data_dir + "岡山天気20227to9.csv",     # 2022年7月から9月までのデータ
    data_dir + "岡山天気202210to12.csv",  # 2022年10月から12月までのデータ
    data_dir + "岡山天気20231to3.csv"     # 2023年1月から3月までのデータ
]

# 各ファイルに対してスキップする行数を指定
#
skiprows_dict = {f: 4 for f in file_list_shikoku + file_list_chugoku}

"""
 ファイルごとにskip数を指定する場合の例
skiprows_dict = {
    "高松天気2022-04to06.csv": 3,
    "高松天気20227to9.csv": 4,
    "岡山天気2022-04to06.csv": 2,
    "岡山天気20227to9.csv": 5
}
"""
# 四国の天気データを読み込み（共通の関数 load_and_concat_multiple を使用）
weather_shikoku = load_and_concat_multiple(file_list_shikoku, region_name="四国", skiprows_dict=skiprows_dict)

# 中国の天気データを読み込み（共通の関数 load_and_concat_multiple を使用）
weather_chugoku = load_and_concat_multiple(file_list_chugoku, region_name="中国", skiprows_dict=skiprows_dict)

# 四国と中国のデータを統合して1つのデータフレームにまとめる
weather_all = pd.concat([weather_shikoku, weather_chugoku]).sort_index()

# 結果の確認表示
print("\n[✅ 四国]")
print(weather_shikoku)  # 四国の天気データ表示

print("\n[✅ 中国]")
print(weather_chugoku)  # 中国の天気データ表示

print("\n[✅ 統合]")
print(weather_all)  # 四国と中国を統合したデータ表示



[✅ 四国]
                     降水量(mm)  風速(m/s)   天気  気温(℃) identifier
年月日時                                                        
2022-04-01 01:00:00      0.5      2.7  4.0    9.8         四国
2022-04-01 02:00:00      0.0      3.2  4.0    9.7         四国
2022-04-01 03:00:00      0.0      5.3  4.0    9.2         四国
2022-04-01 04:00:00      0.0      4.0  4.0    8.7         四国
2022-04-01 05:00:00      0.0      3.7  4.0    8.4         四国
...                      ...      ...  ...    ...        ...
2023-03-31 20:00:00      0.0      2.0  1.0   15.8         四国
2023-03-31 21:00:00      0.0      2.4  1.0   14.5         四国
2023-03-31 22:00:00      0.0      1.2  1.0   14.3         四国
2023-03-31 23:00:00      0.0      0.8  1.0   13.2         四国
2023-04-01 00:00:00      0.0      0.9  1.0   11.9         四国

[8760 rows x 5 columns]

[✅ 中国]
                     降水量(mm)  風速(m/s)   天気  気温(℃) identifier
年月日時                                                        
2022-04-01 01:00:00      0.0      5.7  4.0  

In [22]:
#電力のデータ読み込む
def load_and_prepare_electric_data(filepath, region_name="地域名", skiprows=1):
    """
    電力データを読み込み、前処理を行い、適切な形式で返す関数
    引数:
    - filepath: 読み込むファイルのパス
    - region_name: 地域名（データに追加される）
    - skiprows: 最初にスキップする行数（デフォルト: 1）
    
    戻り値:
    - 前処理されたデータフレーム
    """
    
    # ファイルの拡張子を確認
    ext = os.path.splitext(filepath)[1].lower()
    
    # 拡張子によって読み込み方法を分ける
    if ext == ".csv":
        df = pd.read_csv(filepath, encoding="shift_jis", skiprows=skiprows)  # CSVファイルの読み込み
    elif ext in [".xlsx", ".xls"]:
        df = pd.read_excel(filepath, skiprows=8)  # Excelファイルの読み込み
    else:
        raise ValueError(f"未対応のファイル形式: {ext}")  # 対応していないファイル形式の場合はエラーを発生
    
    # 欠損値をゼロで埋める
    df = df.fillna(0)

    # 日時列の処理
    if "DATE" in df.columns and "TIME" in df.columns:
        # DATEとTIME列を結合して、Datetime型のtime列を作成
        df["time"] = pd.to_datetime(df["DATE"].astype(str) + " " + df["TIME"].astype(str), errors="coerce")
        df = df.set_index("time")  # time列をインデックスに設定
        df = df.drop(columns=["DATE", "TIME"], errors="ignore")  # DATEとTIME列は削除
    elif "年月日時" in df.columns:
        # "年月日時"列があれば、それをDatetime型に変換
        df["time"] = pd.to_datetime(df["年月日時"], errors="coerce")
        df = df.set_index("time")  # time列をインデックスに設定
        df = df.drop(columns=["年月日時"], errors="ignore")  # "年月日時"列は削除
    else:
        # 日時列が見つからなければエラー
        raise ValueError(f"{filepath} に日時列 (DATE+TIME または 年月日時) が存在しません")

    # インデックスが DatetimeIndex であることを確認
    if not isinstance(df.index, pd.DatetimeIndex):
        raise TypeError(f"{filepath} のインデックスが DatetimeIndex ではありません")

    # 地域名をデータフレームに追加
    df["identifier"] = region_name
    
    # "identifier"列を最初に配置
    cols = ["identifier"] + [col for col in df.columns if col != "identifier"]
    df = df[cols]

    return df


In [ ]:
def load_and_concat_multiple(files, region_name, skiprows=1):
    """
    複数の電力データファイルを読み込み、連結する関数
    
    引数:
    - files: 読み込むファイルのリスト
    - region_name: 地域名（各データに付加される）
    - skiprows: 最初にスキップする行数（デフォルト: 1）
    
    戻り値:
    - 複数のデータを連結したデータフレーム
    """
    # 各ファイルを読み込んでリストに格納
    dfs = [load_and_prepare_electric_data(f, region_name, skiprows=skiprows) for f in files]
    
    # データフレームを縦に連結
    combined = pd.concat(dfs).sort_index()  # インデックスでソート
    
    # 重複するインデックスを削除
    combined = combined[~combined.index.duplicated()]

    return combined


In [ ]:
def process_shikoku_data(files, region_name="四国"):
    """
    四国の電力データを処理し、「火力」列を追加する関数

    引数:
    - files: 四国の電力データファイルのリスト
    - region_name: 地域名（デフォルト: 四国）

    戻り値:
    - 処理後のデータフレーム
    """
    # 複数のファイルを読み込み、データフレームを結合する
    df = load_and_concat_multiple(files, region_name, skiprows=1)

    # 火力発電に関する列を抽出（列名に「火力(」が含まれているもの）
    fire_cols = [col for col in df.columns if "火力(" in col]
    
    # 「火力」列を新たに作成し、火力発電の列を合計する
    df["火力"] = df[fire_cols].sum(axis=1)
    
    # 火力発電の個別列は削除
    df = df.drop(columns=fire_cols)

    # インデックスの名前を「年月日時」に変更
    df.index.name = "年月日時"
    
    return df


In [ ]:
def process_chugoku_data(files, region_name="中国"):
    """
    中国の電力データを処理し、列名変更、全角マイナス処理、数値変換を行う関数

    引数:
    - files: 中国の電力データファイルのリスト
    - region_name: 地域名（デフォルト: 中国）

    戻り値:
    - 処理後のデータフレーム
    """
    # 列名変更マッピング
    COLUMN_RENAME_MAP = {
        "需要": "エリア需要",
        "太陽光(実績)": "太陽光発電実績",
        "太陽光(抑制量)": "太陽光出力制御量",
        "風力(実績)": "風力発電実績",
        "風力(抑制量)": "風力出力制御量",
        "連系線潮流": "連系線"
    }

    # 複数のファイルを読み込み、データフレームを結合する
    df = load_and_concat_multiple(files, region_name, skiprows=2)
    
    # 中国のデータで「蓄電池」列の値を0に設定
    df.loc[df["identifier"] == "中国", "蓄電池"] = 0

    # 列名を変更
    df = df.rename(columns=COLUMN_RENAME_MAP)

    # 全角マイナス（−）を0に置き換え
    df = df.replace("−", 0)

    # 数値に変換（数値変換できるものだけ変換、できない場合はそのまま）
    df = df.apply(pd.to_numeric, errors="ignore")

    # インデックスが DatetimeIndex でないと resample できないことを確認
    # if not isinstance(df.index, pd.DatetimeIndex):
    #     raise ValueError("Index must be DatetimeIndex for resampling")

    # 補間処理（コメントアウトされているが、必要であれば有効化）
    # df = df.resample("30T").interpolate()

    # identifier を地域名で補完（省略されている場合）
    # df["identifier"] = region_name

    # インデックスの名前を「年月日時」に変更
    df.index.name = "年月日時"
    
    return df


In [ ]:
# 四国と中国のデータファイル一覧
file_list_shikoku = [
    # ここでは四国のデータファイルを指定
    # data_dir + "四国2505.csv",  # 他のデータファイルも追加可能
    # data_dir + "四国需給.csv",    # ここを適宜修正
    data_dir + "四国需給2022-04.xlsx"  # 四国のデータ（2022年4月）を指定
]

file_list_chugoku = [
    data_dir + "中国2022-04需要.csv",  # 中国のデータ（2022年4月）を指定
]

# 四国のデータを処理する関数の呼び出し
electric_shikoku = process_shikoku_data(file_list_shikoku)

# 中国のデータを処理する関数の呼び出し
electric_chugoku = process_chugoku_data(file_list_chugoku)

# 四国と中国のデータを縦に連結
electric_all = pd.concat([electric_shikoku, electric_chugoku], axis=0)

# インデックス（年月日時）でソート
electric_all = electric_all.sort_index()

# データ中の全角マイナス（−）をゼロ（0）に置換
electric_all = electric_all.replace("－", 0)

# 数値型への変換（数値として変換できる列は数値型に変換）
electric_all = electric_all.apply(pd.to_numeric, errors='ignore')

# インデックスが欠損している行を削除
electric_all = electric_all[electric_all.index.notna()]

# 不要な列（「合計」「その他」）を削除
electric_all = electric_all.drop(columns=["合計", "その他"], errors='ignore')

# 蓄電池列の欠損値を0で補充
electric_all["蓄電池"] = electric_all["蓄電池"].fillna(0)

# データ処理後の結果を確認
# 四国のデータ
print("四国の電力データ:")
print(electric_shikoku)

# 中国のデータ
print("中国の電力データ:")
print(electric_chugoku)

# 統合された電力データ
print("統合された電力データ:")
print(electric_all)

# 統合データの欠損値のカウント
print("欠損値のカウント:")
print(electric_all.isnull().sum())


In [ ]:
# ==============================================
# 電力データと天気データの結合処理
# ==============================================

# ポイント
# 「年月日時」と「identifier（地域名）」の両方をキーにして、
# 電力データ（electric_all）と天気データ（weather_all）を結合します。
# inner join を指定しているため、両方に存在する日時＋地域のデータのみ残ります。

merged_df = electric_all.join(
    weather_all.reset_index().set_index(["年月日時", "identifier"]),  # 天気データ側を multi-index 化
    how="inner",          # 内部結合（両方に共通するキーのみ結合）
    on=["年月日時", "identifier"]  # 結合キー（日時と地域）
)

# ✅ 結合後のデータ確認
print("==== 結合後のデータ（電力 × 天気） ====")
print(merged_df)


In [ ]:
# ==============================================
# スポット価格データ（中国エリア）の読み込みと確認
# ==============================================
# スポット価格データ（2022年）を読み込み
# - ファイルは Shift_JIS でエンコードされている
# - skiprows=0 なので、先頭行をそのまま読み込み
spot = pd.read_csv(
    data_dir + "spot2022to2023.csv",
    encoding="shift_jis",
    skiprows=0
)

# データの中身を確認（全体表示）
print("==== スポット価格データ（2022年から2023年） ====")
print(spot)

# 必要に応じて、上位数行だけ確認
# print(spot.head())

# 列名とデータ型の確認
# print(spot.info())

# 欠損値の確認
# print("欠損値の数:")
# print(spot.isnull().sum())


In [ ]:

# -------------------------------------------------------
# 1. 必要な列のみ抽出
# -------------------------------------------------------
df = spot[["受渡日", "時刻コード", "エリアプライス四国(円/kWh)", "エリアプライス中国(円/kWh)"]].copy()

# -------------------------------------------------------
# 2. 日付・時間の整形
# -------------------------------------------------------
# 「受渡日」を datetime 型に変換
df["受渡日"] = pd.to_datetime(df["受渡日"])

# 時刻コード（1～48）を整数に変換
df["時刻コード"] = df["時刻コード"].astype(int)

# 1時間単位に変換：1,2 → 1時、3,4 → 2時 ... 47,48 → 24時
# （JEPXでは30分単位なので、2コードで1時間分を表す）
df["hour"] = (df["時刻コード"] + 1) // 2

# 「年月日時」列を作成（受渡日 + 時刻）
df["datetime"] = df["受渡日"] + pd.to_timedelta(df["hour"] - 1, unit="h")


# -------------------------------------------------------
# 3. 1時間単位に平均化
# -------------------------------------------------------
# 四国・中国エリアそれぞれで、1時間ごとの平均価格を算出
df_avg = (
    df.groupby("datetime")[["エリアプライス四国(円/kWh)", "エリアプライス中国(円/kWh)"]]
    .mean()
    .reset_index()
)


# -------------------------------------------------------
# 4. wide → long 形式に変換
# -------------------------------------------------------
# "datetime" を基準に、エリアごとの価格を縦持ち化
df_long = df_avg.melt(
    id_vars="datetime",
    value_vars=["エリアプライス四国(円/kWh)", "エリアプライス中国(円/kWh)"],
    var_name="identifier",
    value_name="price"
)

# 列名からエリア名だけ抽出（例：エリアプライス四国 → 四国）
df_long["identifier"] = df_long["identifier"].str.extract(r"エリアプライス(四国|中国)")


# -------------------------------------------------------
# 5. 列名を統一・並び替え
# -------------------------------------------------------
df_long = df_long.rename(columns={
    "datetime": "年月日時",
    "price": "価格(円/kWh)"
})

# 時間と地域でソート
df_long = df_long.sort_values(["年月日時", "identifier"]).reset_index(drop=True)


# -------------------------------------------------------
# 6. 結果確認
# -------------------------------------------------------
print(df_long.head(10))


In [ ]:
# =====================================
# 電力需給・気象・価格データの統合処理
# =====================================
# - これまで作成した電力需給データ (electric_all)
# - 気象データ (weather_all)
# - スポット価格データ (df_long)
# を、「年月日時」と「identifier（地域名）」で結合して一体化する

# -------------------------------------------------------
# 1. 需給＋気象データ（merged_df）に価格データを結合
# -------------------------------------------------------
merged_all = merged_df.join(
    df_long.reset_index().set_index(["年月日時", "identifier"]),
    how="inner",  # 共通部分のみ結合（欠損のないデータを作る）
    on=["年月日時", "identifier"]
)

# 不要な index 列を削除（存在しない場合は無視）
merged_all = merged_all.drop(columns=["index"], errors='ignore')


# -------------------------------------------------------
# 2. 純供給（net supply）の定義
# -------------------------------------------------------
# 「合計」列の代わりに、「エリア需要」をベースに以下で算出：
#   純供給 = エリア需要 - 連系線 - 揚水
# ※連系線：地域外との電力融通（プラスで流入、マイナスで流出）
# ※揚水：夜間に電力を使って水をくみ上げる（実質的に電力消費扱い）

merged_all["純供給"] = (
    merged_all["エリア需要"]
    - merged_all["連系線"]
    - merged_all["揚水"]
)


# -------------------------------------------------------
# 3. 結果確認
# -------------------------------------------------------
print(merged_all.head(10))


In [ ]:
# ==========================================
# 時系列データ識別子（identifier）生成関数
# ==========================================
def create_identifiers(df, region_col='region', datetime_col=None,
                       freq='30min', input_days=2, slide_days=1,
                       target_regions=None):
    """
    特定の地域ごとに、「地域_開始時刻」形式の identifier を自動生成する。
    TFT や LSTM などの時系列モデルにおけるスライディングウィンドウ学習で使用。

    Parameters
    ----------
    df : pandas.DataFrame
        地域と時系列を含むデータフレーム。
    region_col : str, default='region'
        地域名を格納した列の名前。
    datetime_col : str or None, default=None
        日時を格納した列の名前。None の場合は index を日時とみなす。
    freq : str, default='30min'
        データの時間間隔（例：'30min'、'1H'、'15min'）。
    input_days : int, default=2
        1系列の入力に含める日数。たとえば2日の場合、30分間隔なら 96 ステップ。
    slide_days : int, default=1
        スライディングウィンドウの移動幅（日数単位）。1なら半分重複（例：48ステップ）。
    target_regions : list[str] or None, default=None
        特定の地域のみを対象にする場合のリスト。Noneなら全地域を対象。

    Returns
    -------
    identifiers : list[str]
        各地域ごとの「地域_開始時刻」形式の識別子リスト。
        例：["四国_202204010000", "四国_202204020000", "中国_202204010000", ...]
    """

    # ----------------------------------------------
    # 基本設定とソート処理
    # ----------------------------------------------
    df = df.copy()

    if datetime_col is None:
        # indexを日時として扱う場合
        df.index = pd.to_datetime(df.index)
        df = df.sort_index()
    else:
        # 明示的な日時列がある場合
        df[datetime_col] = pd.to_datetime(df[datetime_col])
        df = df.sort_values([region_col, datetime_col])

    # ----------------------------------------------
    # 時系列のステップ計算
    # ----------------------------------------------
    step_per_day = int(pd.Timedelta("1D") / pd.Timedelta(freq))  # 1日あたりのデータ点数
    sequence_length = int(input_days * step_per_day)             # 1入力系列の長さ
    slide_step = max(int(slide_days * step_per_day), 1)          # スライド幅

    identifiers = []

    # ----------------------------------------------
    # 対象地域の決定
    # ----------------------------------------------
    regions = target_regions if target_regions is not None else df[region_col].unique()

    # ----------------------------------------------
    # 各地域ごとにスライディング識別子を作成
    # ----------------------------------------------
    for region in regions:
        region_df = df[df[region_col] == region]
        n = len(region_df)

        for start in range(0, n - sequence_length + 1, slide_step):
            # 開始時刻の取得
            if datetime_col is None:
                start_time_str = region_df.index[start].strftime('%Y%m%d%H%M')
            else:
                start_time_str = region_df.iloc[start][datetime_col].strftime('%Y%m%d%H%M')

            # 識別子作成：例）"四国_202204010000"
            identifier = f"{region}_{start_time_str}"
            identifiers.append(identifier)

    return identifiers


In [ ]:
# ===============================================
#  統合済みデータ（電力＋天気＋価格）の構造確認
# ===============================================

# 各列名の一覧を出力（データ統合が正しく行われたか確認）
print(" [Columns in merged_all]")
print(merged_all.columns)

# インデックス情報を出力（時系列構造・DatetimeIndexの確認）
print("\n [Index of merged_all]")
print(merged_all.index)


In [ ]:
# ===============================================
#  特徴量・目的変数の正規化処理
# ===============================================

# 対象：
# - 数値スケールが大きく異なる項目を 0〜1 に正規化（Min-Max法）
# - TFT や LSTM モデルの学習安定性を高めるための前処理

# -------------------------------------------------------
# 1. 正規化対象のカラム群（電力需給・気象・価格など）
# -------------------------------------------------------
lab = [
    "エリア需要", "原子力", "火力", "水力", "地熱", "バイオマス",
    "太陽光発電実績", "太陽光出力制御量", "風力発電実績", "風力出力制御量",
    "揚水", "連系線", "蓄電池", "降水量(mm)", "風速(m/s)", "気温(℃)", "価格(円/kWh)"
]

# -------------------------------------------------------
# 2. 目的変数（ターゲット）カラム
# -------------------------------------------------------
target = [
    "エリア需要", "価格(円/kWh)", "純供給"
]

# -------------------------------------------------------
# 3. 欠損値の補完
# -------------------------------------------------------
# 時系列データでは、前後の値で補間することが一般的。
# ffill：直前の値で埋める（forward fill）
# bfill：直後の値で埋める（backward fill）
merged_all[lab + ["純供給"]] = merged_all[lab + ["純供給"]].fillna(method="ffill").fillna(method="bfill")

<h1>正規化手法3種類</h1>


In [ ]:


# -------------------------------------------------------
# . Min-Max スケーリング（0〜1範囲）
# -------------------------------------------------------
from sklearn.preprocessing import MinMaxScaler
import joblib

minmaxscaler = MinMaxScaler()
merged_all[lab + ["純供給"]] = minmaxscaler.fit_transform(merged_all[lab + ["純供給"]])

# -------------------------------------------------------
# 5.スケーラーの保存（再現性確保）
# -------------------------------------------------------
# → 将来の推論時に同じスケーリングを再現できるように保存
joblib.dump(minmaxscaler, "scaler_lab_target.save")

# -------------------------------------------------------
# 6. 結果確認
# -------------------------------------------------------
print("✅ 正規化完了：lab + target の数値を MinMaxScaler で処理しました")
print(merged_all.head(36))


In [ ]:
"""
# ===============================================
#  特徴量・目的変数の正規化処理（Global Z-score）
# ===============================================

# 対象：
# - 全エリア・全期間の平均・標準偏差でスケーリング
# - 各系列間の相対スケールを維持しながら安定化
# - TFT / LSTM モデルでのベースラインとして推奨


# -------------------------------------------------------
# Global Z-score 正規化（平均0・標準偏差1）
# -------------------------------------------------------
from sklearn.preprocessing import StandardScaler
import joblib
import numpy as np

scaler = StandardScaler()

# 全体平均・標準偏差で正規化（全地域まとめてスケーリング）
merged_all[lab + ["純供給"]] = scaler.fit_transform(merged_all[lab + ["純供給"]])

# -------------------------------------------------------
# . スケーラーの保存（再現性確保）
# -------------------------------------------------------
joblib.dump(allzscorescaler, "scaler_lab_target_zscore.save")

# -------------------------------------------------------
# . 結果確認
# -------------------------------------------------------
means = np.round(merged_all[lab + ["純供給"]].mean(), 3)
stds  = np.round(merged_all[lab + ["純供給"]].std(), 3)

print("✅ 正規化完了：Global Z-score (平均0・標準偏差1) でスケーリングしました")
print("平均値:\n", means)
print("標準偏差:\n", stds)
print("\n正規化後データの一部:")
print(merged_all.head(36))


In [ ]:
"""
# identifier（地域や系列ID）ごとに#z_score正規化標準化
scalers = {}
scaled_dfs = []

for identifier, group in merged_all.groupby("identifier"):  # ← あなたのカラム名に合わせて変更
    scaler = StandardScaler()
    group[lab + ["純供給"]] = scaler.fit_transform(group[lab + ["純供給"]])
    scalers[identifier] = scaler
    scaled_dfs.append(group)

# 連結して戻す
merged_all = pd.concat(scaled_dfs).sort_index()

# 保存
joblib.dump(z.identifiscalers, "scalers_by_identifier.pkl")

print("✅ identifier ごとに z-score 正規化しました")
print(merged_all.head(36))


In [ ]:
# =========================================================
# Identifier（系列ID）の作成
# =========================================================
# 各地域（region）ごとに、時系列データを学習用スライス単位に分割します。
# 本関数では以下の処理を行います：
#   - 各地域（四国・中国）を識別子として管理
#   - index（日時）を用いて連続データを1時間単位で扱う
#   - 過去2日分の履歴（input_days=2）から次の1日を予測するスライド窓を生成
#
# パラメータ：
#   merged_all : 前処理済みの結合済みデータ
#   region_col : 地域（系列）を示すカラム名（例："identifier"）
#   datetime_col : 日時カラム名（Noneならindexを使用）
#   freq : 時系列の粒度（ここでは1時間単位）
#   input_days : モデル入力に使う過去日数
#   slide_days : スライド幅（日単位）
#   target_regions : 解析対象とする地域リスト
#
# 出力：
#   identifiers : 各地域・各スライド窓のデータをまとめたリストまたはDataFrame
# =========================================================

identifiers = create_identifiers(
    merged_all,
    region_col='identifier',
    datetime_col=None,          # ← index（年月日時）をそのまま使用
    freq='1hour',
    input_days=2,
    slide_days=1,
    target_regions=['四国', '中国']
)

# 内容確認（全体表示 or 部分表示）
#print(identifiers)
print(identifiers[:5])  # 長い場合は最初の5件だけを確認


<h1>前処理と共変量の作成</h1>


In [ ]:
# =========================================================
# extract_sequence_matrix
# =========================================================
# 【概要】
#   各系列（sequence_id）ごとに、指定した特徴量（value_col）を
#   リスト化し、時系列の行列形式（系列×時刻）に変換する関数。
#
# 【用途】
#   - 時系列データをモデル入力（sequence形式）に整形するとき
#   - TFTやRNNなど、サンプル×時系列ステップの構造に変換したい場合
#
# 【処理内容】
#   1. group_col（例：sequence_id）でグループ化
#   2. 各グループ内の value_col を list 化
#   3. リストを展開して DataFrame に変換
#   4. 列名を 1, 2, 3, ... のステップ番号にリネーム
#
# 【引数】
#   df : pandas.DataFrame  
#       - sequence_id列と value_col列を含むDataFrame
#   value_col : str  
#       - 抽出したい変数名（例："気温(℃)"、"降水量(mm)" など）
#   group_col : str（default="sequence_id"）  
#       - グループ化に使う列名（通常は sequence_id）
#
# 【戻り値】
#   matrix_df : pandas.DataFrame  
#       - 各行が1系列、列が時系列ステップを表す行列形式のDataFrame
#
# 【例】
#   >>> extract_sequence_matrix(df, value_col="気温(℃)")
#   # 出力：
#   #        1      2      3   ...  N
#   # seq_1  25.4   25.1   24.8 ...
#   # seq_2  22.9   22.5   22.3 ...
# =========================================================

def extract_sequence_matrix(df, value_col, group_col="sequence_id"):
    """
    各系列（group_col）ごとに value_col をリスト化し、時系列行列に展開する。
    """
    # グループ化して指定列をリスト化
    grouped = df.groupby(group_col)[value_col].apply(list)

    # リストをDataFrameに変換（行＝系列、列＝時系列ステップ）
    matrix_df = pd.DataFrame(grouped.tolist(), index=grouped.index)

    # 列名を1, 2, 3, ... のステップ番号にリネーム
    matrix_df.columns = [i + 1 for i in range(matrix_df.shape[1])]

    return matrix_df


In [ ]:
# =========================================================
# ⚙️ generate_sequence_matrices
# =========================================================
# 【概要】
#   指定した地域 × 開始時刻（identifier）ごとに、指定変数を
#   「系列 × 時刻」の行列形式に展開する関数。
#
# 【用途】
#   - TFT / LSTM / Transformer などの時系列モデル入力データ作成
#   - 変数ごとに sequence_id を持つサンプル行列を自動生成
#
# 【処理内容】
#   1. identifier ("地域_YYYYMMDDHHMM") から地域と開始時刻を抽出
#   2. 開始時刻〜sequence_length時間分のデータを切り出し
#   3. 各サンプルに sequence_id を付与して結合
#   4. extract_sequence_matrix() を使って各変数を行列化
#
# 【引数】
#   df : pandas.DataFrame  
#       - 元の時系列データ（datetime index, id_colを含む）
#   identifiers : list[str]  
#       - "地域_YYYYMMDDHHMM" の形式で定義された系列IDのリスト
#   variables : list[str]  
#       - 対象とする数値カラム（例："気温(℃)"、"価格(円/kWh)" など）
#   sequence_length : int  
#       - 1系列あたりの時間ステップ数（例：36 → 36時間分）
#   id_col : str（default="identifier"）  
#       - 地域やエリアを示す識別子の列名
#   datetime_index_name : str（default="年月日時"）  
#       - インデックスとして扱う日時カラム名
#
# 【戻り値】
#   matrix_dict : dict[str, DataFrame]  
#       - 各変数ごとの行列（行=系列、列=時刻ステップ）
#   merged_with_seq : pandas.DataFrame  
#       - 元データに sequence_id を付与した結合済みDataFrame
#
# 【例】
#   >>> matrix_dict, merged_with_seq = generate_sequence_matrices(
#           df=merged_all,
#           identifiers=identifiers,
#           variables=["気温(℃)", "エリア需要"],
#           sequence_length=36
#       )
#   >>> matrix_dict["気温(℃)"].shape
#   (N系列, 36)
# =========================================================

def generate_sequence_matrices(
    df,
    identifiers,
    variables,
    sequence_length=36,
    id_col="identifier",
    datetime_index_name="年月日時"
):
    """
    指定した identifier リストに基づき、各変数を sequence × time の行列に変換する。
    """
    df = df.copy()
    df.index.name = datetime_index_name  # 念のためインデックス名を指定

    sequence_records = []

    # =========================================================
    # 各 identifier ("地域_YYYYMMDDHHMM") に対して
    # 開始〜終了時刻のデータを抽出し、sequence_idを付与
    # =========================================================
    for ident in identifiers:
        region, start_str = ident.split("_")
        start_time = pd.to_datetime(start_str, format="%Y%m%d%H%M")
        end_time = start_time + pd.Timedelta(hours=sequence_length - 1)

        # 指定範囲＆地域のデータを抽出
        temp_df = df[
            (df.index >= start_time) &
            (df.index <= end_time) &
            (df[id_col] == region)
        ].copy()

        temp_df["sequence_id"] = ident
        sequence_records.append(temp_df)

    # =========================================================
    # すべてのサンプルを結合
    # =========================================================
    merged_with_seq = pd.concat(sequence_records)

    # =========================================================
    # 各変数の系列行列を生成（extract_sequence_matrix を利用）
    # =========================================================
    matrix_dict = {}
    for col in variables:
        try:
            matrix_df = extract_sequence_matrix(
                merged_with_seq,
                value_col=col,
                group_col="sequence_id"
            )
            matrix_dict[col] = matrix_df
            print(f"✅ {col}: {matrix_df.shape}")
        except KeyError:
            print(f"⚠️ カラムが見つかりませんでした: {col}")

    return matrix_dict, merged_with_seq


# =========================================================
# 💡 使用例
# =========================================================
lab = [
    "エリア需要", "原子力", "火力", "水力", "地熱", "バイオマス",
    "太陽光発電実績", "太陽光出力制御量", "風力発電実績", "風力出力制御量",
    "揚水", "連系線", "蓄電池", "降水量(mm)", "風速(m/s)", "気温(℃)", "価格(円/kWh)"
]

matrix_dict, merged_with_seq = generate_sequence_matrices(
    df=merged_all,
    identifiers=identifiers,
    variables=lab,
    sequence_length=36  # 🔁 36時間系列（過去2日分など）
)

# 各変数の行列を個別に取得
temp_df       = matrix_dict["気温(℃)"]
rain_df       = matrix_dict["降水量(mm)"]
wind_df       = matrix_dict["風速(m/s)"]
demand_df     = matrix_dict["エリア需要"]
nuclear_df    = matrix_dict["原子力"]
thermal_df    = matrix_dict["火力"]
hydro_df      = matrix_dict["水力"]
geothermal_df = matrix_dict["地熱"]
biomass_df    = matrix_dict["バイオマス"]
solar_df      = matrix_dict["太陽光発電実績"]
solar_ctrl_df = matrix_dict["太陽光出力制御量"]
windgen_df    = matrix_dict["風力発電実績"]
windctrl_df   = matrix_dict["風力出力制御量"]
pump_df       = matrix_dict["揚水"]
tie_df        = matrix_dict["連系線"]
battery_df    = matrix_dict["蓄電池"]
price_df      = matrix_dict["価格(円/kWh)"]

print(temp_df)


In [ ]:
# =========================================================
# 📦 Dynamic Covariates の行列生成
# =========================================================
# 【目的】
#   - 電力需要・供給・価格の予測に使用する
#     “動的共変量（過去の発電・気象・設備データなど）” を
#     TFT などの時系列モデルが扱える形（系列 × 時間ステップ）に変換する。
#
# 【特徴】
#   - 36ステップ（=36時間分）の連続データをスライド窓で抽出
#   - 各地域×開始時刻に sequence_id を割り当て
#   - 各変数ごとに (系列 × 時間) の行列を自動生成
#
# 【対象変数】
#   - 発電関連: 原子力, 火力, 水力, 地熱, バイオマス
#   - 再エネ関連: 太陽光, 風力, 出力制御量
#   - 系統関連: 揚水, 連系線, 蓄電池
#   - 気象情報: 降水量, 風速, 気温
# =========================================================

lab = [
    "原子力", "火力", "水力", "地熱", "バイオマス",
    "太陽光発電実績", "太陽光出力制御量", "風力発電実績", "風力出力制御量",
    "揚水", "連系線", "蓄電池", "降水量(mm)", "風速(m/s)", "気温(℃)"
]

# =========================================================
# 🧩 Sequence Matrix 生成
# =========================================================
matrix_dict, merged_with_seq = generate_sequence_matrices(
    df=merged_all,
    identifiers=identifiers,   # 地域×開始時刻リスト（"四国_202204010000"など）
    variables=lab,             # 動的共変量リスト
    sequence_length=36         # 🔁 過去36時間を1系列とする
)

# 各変数の shape 確認（例: (


In [ ]:
# 気温・降水量・風速・発電量など、各変数ごとのシーケンス行列を取得
# （matrix_dict は generate_sequence_matrices() で生成された辞書）
# 各 DataFrame は identifier（地域など）× 時系列シーケンス（36時間分）を格納
# 例：temp_df は「気温(℃)」の36時間分の入力系列を表す
temp_df       = matrix_dict["気温(℃)"]
rain_df       = matrix_dict["降水量(mm)"]
wind_df       = matrix_dict["風速(m/s)"]
nuclear_df    = matrix_dict["原子力"]
thermal_df    = matrix_dict["火力"]
hydro_df      = matrix_dict["水力"]
geothermal_df = matrix_dict["地熱"]
biomass_df    = matrix_dict["バイオマス"]
solar_df      = matrix_dict["太陽光発電実績"]
solar_ctrl_df = matrix_dict["太陽光出力制御量"]
windgen_df    = matrix_dict["風力発電実績"]
windctrl_df   = matrix_dict["風力出力制御量"]
pump_df       = matrix_dict["揚水"]
tie_df        = matrix_dict["連系線"]
battery_df    = matrix_dict["蓄電池"]

# 出力確認（例：気温の系列行列を表示）
print(temp_df.head())


In [ ]:
# ===============================================================
#  複数変数の時系列行列を 3次元テンソル [系列 × 時間 × 特徴量] に統合
# ---------------------------------------------------------------
# 各変数（例：気温・降水量・発電量など）は 2次元行列（系列×時間）として
# matrix_dict に格納されているため、それらを1つのテンソルにまとめる。
# この形式は PyTorch や TensorFlow などの入力形式に適している。
# ===============================================================

# 1つ目の変数から、系列数（num_sequences）と時系列長（sequence_length）を取得
first_var = lab[0]
num_sequences, sequence_length = matrix_dict[first_var].shape

# 特徴量（変数）の数
num_features = len(lab)

# 空の3次元テンソルを作成：[系列数, 時系列長, 特徴量数]
labs = np.zeros((num_sequences, sequence_length, num_features), dtype=np.float32)

# 各変数をテンソルに順番に格納
for i, var in enumerate(lab):
    try:
        labs[:, :, i] = matrix_dict[var].values
    except Exception as e:
        print(f"⚠️ 変数 {var} の埋め込み中にエラー発生: {e}")

# 確認出力
print(labs)          # テンソルの中身（大きいので必要に応じて削除）
print(labs.shape)    # => (系列数, 時間長, 特徴量数)


In [ ]:
# ===============================================================
# 🔹 静的共変量（Static Covariates）の生成
# ---------------------------------------------------------------
# 各 sequence_id（例："四国_202106010000"）から地域・日時情報を抽出し、
# 地域の人口や時間的特徴（月・日・曜日）などを静的特徴量として付与。
# TFT などのモデルで「系列ごとの固定情報」を扱うために利用。
# ===============================================================

# ---------------------------------------------------------------
# 2.パラメータ設定
# ---------------------------------------------------------------
past_months = 36      # 過去の系列長（例：36時間分）
future_months = 12    # 未来予測ステップ数（例：12時間分）

# 各系列のID（例："四国_202106010000"）を取得
sequence_ids = temp_df.index.to_series()

# ---------------------------------------------------------------
# 2. sequence_id から 地域 と 開始日時 を抽出
# ---------------------------------------------------------------
region = sequence_ids.str.split("_").str[0]
start_time = pd.to_datetime(sequence_ids.str.split("_").str[1], format="%Y%m%d%H%M")

# ---------------------------------------------------------------
# 3地域ごとの人口データ（静的な定量情報）
# ---------------------------------------------------------------
population_dict = {
    "中国": 72500,   
    "四国": 37000,
}

# ---------------------------------------------------------------
# 4.静的情報DataFrameを構築
# ---------------------------------------------------------------
static_df = pd.DataFrame({
    "sequence_id": sequence_ids.values,
    "地域": region.values,
    "人口": region.map(population_dict).values,
    "月": start_time.dt.month.values,
    "日": start_time.dt.day.values,
    "曜日": start_time.dt.weekday.values,  # 月曜=0〜日曜=6
}).set_index("sequence_id")

# ---------------------------------------------------------------
# 5. 地域を One-Hot エンコーディング化
# ---------------------------------------------------------------
region_onehot = pd.get_dummies(static_df["地域"], prefix="地域", dtype=int)

# ---------------------------------------------------------------
# 6.数値情報とカテゴリ情報を統合した静的特徴DataFrameを作成
# ---------------------------------------------------------------
static = pd.concat([
    static_df[["人口", "月", "日", "曜日"]],
    region_onehot
], axis=1)

# ---------------------------------------------------------------
# 7. 数値型・カテゴリ型の静的共変量を別々に抽出
# ---------------------------------------------------------------
# 数値的静的共変量（例：人口・月・日・曜日）
static_numeric = static_df[["人口", "月", "日", "曜日"]].astype(float).values

# カテゴリ的静的共変量（例：地域のOne-Hot表現）
static_categoric = region_onehot.values

# ✅ 出力例
# static_numeric.shape → (系列数, 4)
# static_categoric.shape → (系列数, 地域数)


In [ ]:
# ===============================================================
# 🔹 時間特徴行列（Time Index Matrix）の構築
# ---------------------------------------------------------------
# 各系列（sequence_id）に対応する「時刻系列（datetime配列）」を生成。
# 各行が1系列、各列が1時間ステップ（例：48時間）を表す。
# TFTやLSTMなどで時間情報を補助特徴量として使う際に有用。
# ===============================================================

# ---------------------------------------------------------------
# 1. シーケンス長と時間オフセットを設定
# ---------------------------------------------------------------
seq_len = 48  # 1系列あたりの時間ステップ数（例：48時間）

# 各時間ステップのオフセットを Timedelta で生成
hour_offsets = pd.to_timedelta(np.arange(seq_len), unit="h")

# ---------------------------------------------------------------
# 2. 各系列の開始時刻を sequence_id から抽出
# ---------------------------------------------------------------
sequence_ids = temp_df.index.to_series()  # ex. "四国_202204010000"
start_times = pd.to_datetime(
    sequence_ids.str.split("_").str[1],
    format="%Y%m%d%H%M"
)

# ---------------------------------------------------------------
# 3. 各系列の時刻配列を構築（行＝系列，列＝時間ステップ）
# ---------------------------------------------------------------
time_matrix = np.array([
    start_time + hour_offsets
    for start_time in start_times
])

# ---------------------------------------------------------------
# 4. DataFrame 形式に変換して可読性を向上
# ---------------------------------------------------------------
time_df = pd.DataFrame(time_matrix, index=sequence_ids)
time_df.columns = list(range(seq_len))  # 列名: 0〜(seq_len-1)
# 例）列0=開始時刻，列47=開始時刻＋47時間

# ---------------------------------------------------------------
# 5. datetime → 整数形式（例：2022040101）へ変換
# ---------------------------------------------------------------
Age = time_df.applymap(lambda dt: int(dt.strftime("%Y%m%d%H")))

# NumPy配列形式に変換（モデル入力用）
age = Age.values

# ✅ 確認：先頭5系列の時間行列を表示
print(Age.head())


In [ ]:
# ============================================================
# 🎯 目的変数（ターゲット系列）の生成
# ------------------------------------------------------------
# 本ブロックでは、TFTモデルで予測対象となる3変数
# 「エリア需要」「価格(円/kWh)」「純供給」
# を sequence_id（地域×開始時刻）ごとに展開し、
# 各変数を [系列 × 時間ステップ] の行列形式に変換します。
# ============================================================

target = [
    "エリア需要",        # 電力需要（消費側）
    "価格(円/kWh)",      # 市場価格（JEPXなど）
    "純供給"             # 純供給（供給総量 - 連系線 - 揚水）
]

# ⚙️ 48時間分の系列を対象に sequence_id 付きデータを作成
target_dict, merged_target_seq = generate_sequence_matrices(
    df=merged_all,
    identifiers=identifiers,
    variables=target,
    sequence_length=48  # 🔁 ここを変えると予測期間を調整できる（例: 24, 36など）
)

# 各ターゲット変数を個別に抽出
demand_df = target_dict["エリア需要"]        # 需要系列（行=系列ID、列=時刻）
price_df  = target_dict["価格(円/kWh)"]      # 価格系列
supply_df = target_dict["純供給"]            # 純供給系列

# ✅ 確認（デバッグ時のみ表示）
print("✔️ ターゲット系列生成完了：")
print(f"需要: {demand_df.shape}, 価格: {price_df.shape}, 純供給: {supply_df.shape}")


In [ ]:
# 欠損処理とマスク作成
# 各ターゲット変数（需要・純供給・価格）について、
# 欠損位置をマスク化し、前方・後方補完で埋める
demand_mask = demand_df.notnull().astype(int).values
demand_filled = demand_df.ffill(axis=1).bfill(axis=1).values

supply_mask = supply_df.notnull().astype(int).values
supply_filled = supply_df.ffill(axis=1).bfill(axis=1).values

price_mask = price_df.notnull().astype(int).values
price_filled = price_df.ffill(axis=1).bfill(axis=1).values


In [ ]:
# 天気カテゴリ変数を時系列シーケンス化
# 各identifier（地域など）ごとに、過去36ステップ分を生成
categori = ["天気"]

glasgow_dict, merged_glasgow_seq = generate_sequence_matrices(
    df=merged_all,
    identifiers=identifiers,
    variables=categori,
    sequence_length=36  # 🔁 シーケンス長を指定（例：36時間分）
)

# "天気" 列を抽出して確認
glasgow = glasgow_dict["天気"]
print(glasgow)


In [ ]:
# ===========================================
# 欠損値補完とデータ確認（静的/動的共変量）
# ===========================================

# Glasgow（天気）変数：NaN は前後値で補完（カテゴリ変数のため最頻値近似）
glasgow_filled = glasgow.ffill(axis=1).bfill(axis=1).values  

# 各目的変数の shape を確認
# 例：（系列数, 時系列長）
print("Shapes:")
print("  demand_mask:", demand_mask.shape)
print("  supply_mask:", supply_mask.shape)
print("  price_mask: ", price_mask.shape)

# NaN が残っていないか確認
print("Remaining NaN count:")
print("  demand:", np.count_nonzero(np.isnan(demand_filled)))
print("  supply:", np.count_nonzero(np.isnan(supply_filled)))
print("  price: ", np.count_nonzero(np.isnan(price_filled)))
print("  weather:", np.count_nonzero(np.isnan(glasgow_filled)))


In [ ]:
# ===========================================
# 3次元テンソル化: (系列数, 総時系列長, 変数数)
# ===========================================
# static_numeric.shape[0] : 系列数（例: 地域×期間）
# past_months + future_months : 1系列あたりの時系列長
# 3 : 予測対象変数（需要・供給・価格）
targets = np.zeros((static_numeric.shape[0], past_months + future_months, 3))

# 各変数を対応スロットに格納
targets[..., 0] = demand_filled
targets[..., 1] = supply_filled
targets[..., 2] = price_filled


# ===========================================
# マスク作成: 予測対象区間 (future_months) のみ抽出
# ===========================================
# TFTの学習では「どの時点が観測済みか」をマスクで管理する
targets_masks = np.zeros((static_numeric.shape[0], future_months, 3))
targets_masks[..., 0] = demand_mask[:, past_months:]
targets_masks[..., 1] = supply_mask[:, past_months:]
targets_masks[..., 2] = price_mask[:, past_months:]


# --- 検証 ---
print("targets shape :", targets.shape)
print("missing count :", np.count_nonzero(np.isnan(targets)))
print("mask shape    :", targets_masks.shape)
print("mask sum      :", targets_masks.sum().sum())

<h1>Dataloderの作成</h1>


In [ ]:
class TimeSeriesDataset(Dataset):
    """
    Temporal Fusion Transformer (TFT-multi) 用のデータセットクラス。

    各サンプル（= 地域ごとの時系列系列）について以下を格納：
    - 静的共変量（地域の人口、季節情報など）
    - 動的共変量（気象条件・発電構成・価格など）
    - 予測対象（需要・純供給・市場価格）
    """

    def __init__(self, static_numeric, static_categoric, labs, age, g_score, target_arr, target_mask):
        # === 静的情報 ===
        # 地域ごとの人口・時間特徴（numeric）および地域カテゴリ（categoric）
        self.static_categorical = static_categoric
        self.static_numerical = static_numeric

        cohort = age.shape[0]  # 系列数（例：地域×期間）

        # === 過去の履歴データ ===
        # 動的共変量（labs）+ 目的変数（target_arr）+ 時刻情報（age）を結合
        self.historical_ts_numeric = np.concatenate((
            labs[:, :past_months, :],                  # 気象・発電・需給などの動的特徴
            target_arr[:, :past_months, :],            # 目的変数（需要・純供給・価格）の過去値
            age[:, :past_months].reshape(cohort, past_months, 1)  # 時刻（インデックス情報）
        ), axis=2)

        # NaN を特徴ごとに平均値で補完（TFT 入力安定化のため）
        for feature_idx in range(self.historical_ts_numeric.shape[2]):
            feature_data = self.historical_ts_numeric[:, :, feature_idx]
            mean_val = np.nanmean(feature_data)
            feature_data[np.isnan(feature_data)] = mean_val
            self.historical_ts_numeric[:, :, feature_idx] = feature_data

        # === カテゴリ型の時系列変数 ===
        # 例: 天気（"晴れ" "雨" など）→ g_score
        self.historical_ts_categorical = g_score[:, :past_months].reshape(cohort, past_months, 1)

        # === 未来の共変量 ===
        # 例: 将来の時刻情報を特徴量として使用（Decoder Input 用）
        self.future_ts_numeric = age[:, past_months:].reshape(cohort, future_months, 1)

        # === 予測対象とマスク ===
        self.target = target_arr[:, past_months:]      # 未来区間の目的変数
        self.target_mask = target_mask                 # 予測区間の観測マスク

        # sequence ID 管理（地域＋時刻など）
        if sequence_ids is None:
            self.sequence_ids = np.arange(target_arr.shape[0]).astype(str)
        else:
            self.sequence_ids = np.asarray(sequence_ids)

        print("historical_ts_numeric の shape:", self.historical_ts_numeric.shape)

    def __len__(self):
        """サンプル数（系列数）を返す"""
        return len(self.target)

    def __getitem__(self, idx):
        """
        1系列分のデータを辞書形式で返す。
        TFT モデルが期待するキー構造に合わせている。
        """
        static_cat = self.static_categorical[idx, ...]
        static_num = self.static_numerical[idx, ...]
        hist_ts_num = self.historical_ts_numeric[idx, ...]
        hist_ts_cat = self.historical_ts_categorical[idx, ...]
        future_ts_num = self.future_ts_numeric[idx, ...]
        target_i = self.target[idx]
        target_mask_i = self.target_mask[idx]

        return {
            'static_feats_categorical': torch.tensor(static_cat, dtype=torch.long),
            'static_feats_numeric': torch.tensor(static_num, dtype=torch.float32),
            'historical_ts_categorical': torch.tensor(hist_ts_cat, dtype=torch.long),
            'historical_ts_numeric': torch.tensor(hist_ts_num, dtype=torch.float32),
            'future_ts_numeric': torch.tensor(future_ts_num, dtype=torch.float32),
            'target': torch.tensor(target_i, dtype=torch.float32),
            'target_mask': torch.tensor(target_mask_i, dtype=torch.int32),
        }


In [ ]:
# ============================================
# 型変換（Type casting）
# --------------------------------------------
# 各入力変数を学習モデルで扱いやすい型に統一する。
# - float32 : ニューラルネットワークでの標準的な数値型（メモリ効率・GPU計算対応）
# - int     : Embedding層などで扱うカテゴリ型に適用
# ============================================

static_numeric = static_numeric.astype(np.float32)   # 静的な数値特徴
static_categoric = static_categoric.astype(int)      # 静的なカテゴリ特徴
labs = labs.astype(np.float32)                       # 動的共変量
age = age.astype(np.float32)                         # 数値特徴
g_score = glasgow_filled.astype(int)                 # カテゴリ的特徴
targets = targets.astype(np.float32)                 # 予測対象
targets_masks = targets_masks.astype(int)            # マスク


In [ ]:
# ============================================
# データ分割（Train / Test Split）
# --------------------------------------------
# 情報漏洩（Data Leakage）を防ぐため、
# 時系列の「最後の期間」をテストデータとして確保。
# 未来の情報が学習データに混入しないよう、
# 各サンプルの開始時刻（age[:, 0]）を基準に並べ替える。
# ============================================

# 各サンプルの最初の時刻（列0）を基準にソート
start_times = age[:, 0]  # shape = (サンプル数,)
sorted_indices = np.argsort(start_times)

# データ数の取得と分割比設定（後ろ10%をテスト用）
N = len(sorted_indices)
test_size = int(N * 0.1)
train_size = N - test_size

# 訓練・テストセットのインデックスを分割
train_set = sorted_indices[:train_size]      # 過去データで学習
test_set = sorted_indices[train_size+2:]     # 未来データで評価（重複回避）

print(f"train: {len(train_set)}, test: {len(test_set)}")
print(test_set)


In [ ]:

#test_set = np.random.choice(age.shape[0], size=int(np.floor(age.shape[0]*0.1)), replace=False)
#train_set = np.setdiff1d(np.arange(age.shape[0]), test_set)
#print(len(train_set), len(test_set))

b_size = 4

train_dataset = TimeSeriesDataset(static_numeric[train_set, :], 
                                  static_categoric[train_set, :], 
                                  labs[train_set, :], 
                                  age[train_set, :], 
                                  g_score[train_set,:],
                                  targets[train_set, :],
                                 targets_masks[train_set,:])
train_dataloader = DataLoader(train_dataset, batch_size=b_size, shuffle=False)

test_dataset = TimeSeriesDataset(static_numeric[test_set, :],
                                  static_categoric[test_set, :],
                                  labs[test_set, :],
                                  age[test_set, :],
                                  g_score[test_set,:],
                                 targets[test_set, :],
                                targets_masks[test_set, :])
test_dataloader = DataLoader(test_dataset, batch_size=b_size, shuffle=False)


In [ ]:
# ============================================
# 時系列の一貫性チェック（情報漏洩の防止確認）
# --------------------------------------------
# 各系列（サンプル）の「最初の時刻」を基準に、
# 訓練データとテストデータの時間的な順序を検証。
# 目的：Train < Test の時系列順が保たれているかを確認する。
# ============================================

# 各系列の最初の時間（列0）を抽出
start_times = age[:, 0]  # shape: (サンプル数,)

# 各セットの開始時刻を取得
train_times = start_times[train_set]
test_times = start_times[test_set]

# 代表的な開始時刻を出力
print("Train max start time:", train_times.max())
print("Test min start time :", test_times.min())

# 時系列順の整合性を検証
if train_times.max() < test_times.min():
    print("✅ Test set is strictly after the train set in time.")
else:
    print("⚠️ Test set may contain samples earlier than or overlapping with train set.")


In [ ]:
# ============================================================
# DataLoader 出力の確認
# ------------------------------------------------------------
# - TimeSeriesDataset クラスの __getitem__() が正しく動作しているかを検証。
# - 1バッチ分の各テンソル形状を表示して、モデル入力と整合しているか確認。
# ============================================================

first = False
for data in train_dataloader:
    if not first:
        # 🧩 各入力テンソルの shape を出力
        # static_feats_categorical : 静的カテゴリ変数
        # static_feats_numeric     : 静的数値変数
        # historical_ts_categorical: 過去のカテゴリ系列
        # historical_ts_numeric    : 過去の数値系列
        # future_ts_numeric        : 未来の共変量
        # target                   : 予測対象
        # target_mask              : マスク
        print(
            data['static_feats_categorical'].shape,
            data['static_feats_numeric'].shape,
            data['historical_ts_categorical'].shape,
            data['historical_ts_numeric'].shape,
            data['future_ts_numeric'].shape,
            data['target'].shape,
            data['target_mask'].shape
        )
        first = True
    else:
        break


<h1>TFTの-multiモデルの学習</h1>


In [ ]:
# ============================================================
# モジュールのインポート（TFT_model）
# ------------------------------------------------------------
# sys.path.append を使用して、特定のディレクトリを Python の検索パスに追加し、TFT_model モジュールをインポート。
# この方法により、他のディレクトリ内にあるモジュールも、プログラム内で利用可能になる。
# ------------------------------------------------------------
import sys

# TFTモデルのパスを追加（ここではTFT-multi_csvディレクトリを指定）
sys.path.append("/home/nishimura/seminar2025/M1/Nishimura/TFT/TFT-multi_csv/")

# TFT_modelをインポート（モデルの定義やトレーニング、評価に使用）
import model as TFT_model


In [ ]:
class QueueAggregator:
    """
    固定サイズのキュー（FIFO: 先入れ先出し）を扱うクラス。
    主に学習中の損失値などを一定数だけ保持して平均化する用途で使用します。
    """

    def __init__(self, max_size: int):
        """
        コンストラクタ
        ----------
        max_size : int
            保持できる要素の最大数（キューの長さ上限）
        """
        self._queued_list = []
        self.max_size = max_size

    def append(self, elem):
        """
        要素をキューに追加。
        max_size を超えた場合、最も古い要素を削除。
        """
        self._queued_list.append(elem)
        if len(self._queued_list) > self.max_size:
            self._queued_list.pop(0)

    def get(self):
        """
        現在のキューの中身をリストとして返す。
        """
        return self._queued_list


In [ ]:
print(data['static_feats_categorical'], data['static_feats_numeric'],
          data['historical_ts_categorical'], data['historical_ts_numeric'], 
          data['future_ts_numeric'], data['target'], data['target_mask'])

In [ ]:
# ============================================================
# Temporal Fusion Transformer-Multi 用のデータプロパティ生成関数
# ------------------------------------------------------------
# `build_data_props_from_batch` 関数は、`TimeSeriesDataset` から取得したバッチデータを基にして、
# Temporal Fusion Transformer (TFT)-multi に必要な `data_props` を自動で生成します。
# これにより、モデルに必要な設定（静的・歴史的・未来の特徴量の数など）を簡単に取得できます。
# ------------------------------------------------------------
def build_data_props_from_batch(batch: Dict[str, torch.Tensor],
                                num_feature_predicted: int) -> Dict[str, object]:
    """
    データローダから取り出した 1 バッチを解析し、
    Temporal Fusion Transformer‑multi 用の data_props を自動生成する。

    Parameters
    ----------
    batch : dict
        TimeSeriesDataset が返す辞書（static/historical/future などを含む）
    num_feature_predicted : int
        予測対象の変数数 (= targets.shape[-1])

    Returns
    -------
    data_props : dict
        configuration['data_props'] にそのまま渡せる辞書
    """
    # ---------- 静的特徴 ----------
    static_num_cols = batch['static_feats_numeric'     ].shape[-1]   # 4
    static_cat_cols = batch['static_feats_categorical' ].shape[-1]   # 2

    # cardinality = 各列の (max + 1)
    static_cat = batch['static_feats_categorical']
    static_card: List[int] = [(static_cat[:, i].max().item() + 1)
                              for i in range(static_cat_cols)]

    # ---------- 過去系列 ----------
    hist_num_cols = batch['historical_ts_numeric'      ].shape[-1]   # 19
    hist_cat_cols = batch['historical_ts_categorical'  ].shape[-1]   # 1

    hist_cat = batch['historical_ts_categorical']
    hist_card: List[int] = [(hist_cat[:, :, i].max().item() + 1)
                            for i in range(hist_cat_cols)]

    # ---------- 未来系列 ----------
    fut_num_cols = batch['future_ts_numeric'           ].shape[-1]   # 1
    # ※今回は future のカテゴリ変数なし想定

    # ---------- まとめ ----------
    data_props = {
        'num_historical_numeric'        : hist_num_cols,
        'num_historical_categorical'    : hist_cat_cols,
        'historical_categorical_cardinalities': hist_card,

        'num_static_numeric'            : static_num_cols,
        'num_static_categorical'        : static_cat_cols,
        'static_categorical_cardinalities'    : static_card,

        'num_future_numeric'            : fut_num_cols,
        'num_feature_predicted'         : num_feature_predicted
    }
    return data_props

# ============================================================
# 使用例
# ------------------------------------------------------------
# `train_dataloader` から最初のバッチを取得し、`build_data_props_from_batch` を実行。
# これにより、TFT-multi 用のデータ設定（特徴量の数やカテゴリ性）を自動的に取得します。
# ------------------------------------------------------------
first_batch = next(iter(train_dataloader))       # １ミニバッチ取得
data_props  = build_data_props_from_batch(first_batch,
                                    num_feature_predicted=first_batch['target'].shape[-1])

# 自動生成した data_props を表示
print("=== 自動生成した data_props ===")
for k, v in data_props.items():
    print(f"{k:<35}: {v}")


In [ ]:
def scan_hist_categorical(loader):
    """
    🔍 scan_hist_categorical(loader)

    学習データローダー（loader）内の全バッチを走査し、
    時系列カテゴリ変数（historical_ts_categorical）の最小値と最大値を求める関数。

    【目的】
    Embedding層で使用するカテゴリ変数の範囲（cardinality）を正確に把握するため。
    PyTorchでは、カテゴリ値がEmbeddingの範囲外（num_embeddingsを超える）になると、
    「CUDA error: device-side assert triggered」が発生するため、事前チェックが重要。

    【処理内容】
    - 全バッチを順に処理し、カテゴリ変数の最大値・最小値を更新。
    - 各バッチから 'historical_ts_categorical' テンソルを取得し、
      その max/min を計算。
    - long型（int64）テンソルを前提として動作。

    【返り値】
    (global_min, global_max)
    → Embeddingに使うときは、cardinality = global_max + 1 とする。

    【使用例】
    hist_min, hist_max = scan_hist_categorical(train_dataloader)
    print(f"[SCAN] hist_cat min={hist_min} , max={hist_max}")
    # 出力例: [SCAN] hist_cat min=0 , max=10
    """

    global_max = -1
    global_min =  1e9
    for b in loader:
        # long型テンソルとして取り出す
        cat = b['historical_ts_categorical']
        global_max = max(global_max, int(cat.max()))
        global_min = min(global_min, int(cat.min()))
    return global_min, global_max


# 実行例
hist_min, hist_max = scan_hist_categorical(train_dataloader)
print(f"[SCAN]   hist_cat min={hist_min} , max={hist_max}")


In [ ]:
# ============================================================
# 🔧 カテゴリ変数の cardinality（埋め込みテーブルサイズ）を自動推定
# ------------------------------------------------------------
# Embedding 層では、カテゴリ変数の最大値 + 1 が必要（0始まりのため）
# → 各カテゴリ列の最大値をスキャンし、テーブルサイズを算出する。
# ============================================================

# ---------- historical（過去系列） ----------
# 時系列カテゴリ変数の最小値・最大値をデータローダ全体から取得
hist_min, hist_max = scan_hist_categorical(train_dataloader)
print(f"[SCAN] hist_cat min={hist_min} , max={hist_max}")

# 0スタートの場合、Embeddingのテーブルサイズ = max + 1
hist_cardinality = hist_max + 1

# data_props に登録
data_props['historical_categorical_cardinalities'] = [hist_cardinality]
data_props['num_historical_categorical']            = 1  # 列数 = 1

# ---------- static（静的特徴） ----------
# 各バッチ内でカテゴリ変数の最小値・最大値を取得し、同様に +1
static_min  = int(first_batch['static_feats_categorical'].min())
static_max  = int(first_batch['static_feats_categorical'].max())
static_card = static_max + 1

# num_static_categorical の列数ぶん複製して登録
data_props['static_categorical_cardinalities'] = [static_card] * \
                                                 data_props['num_static_categorical']

print(f"[INFO] historical_cardinality={hist_cardinality}, static_cardinality={static_card}")


In [ ]:
# ============================================================
# サニティチェック関数
# ------------------------------------------------------------
# `sanity_check` 関数は、バッチデータとプロパティ（`props`）を受け取り、形状や定義が一致することを確認するためのチェックを行います。
# これにより、データローダーの出力が期待通りの形式であることを保証します。
# ------------------------------------------------------------
def sanity_check(batch, props):
    # 静的カテゴリ特徴量の次元確認
    assert batch['static_feats_categorical'].shape[-1] == props['num_static_categorical']
    assert len(props['static_categorical_cardinalities']) == props['num_static_categorical']

    # 歴史的カテゴリ特徴量の次元確認
    assert batch['historical_ts_categorical'].shape[-1] == props['num_historical_categorical']
    assert len(props['historical_categorical_cardinalities']) == props['num_historical_categorical']

    # 歴史的数値特徴量の次元確認
    assert batch['historical_ts_numeric'].shape[-1] == props['num_historical_numeric']
    
    # 将来の数値特徴量の次元確認
    assert batch['future_ts_numeric'].shape[-1] == props['num_future_numeric']
    
    # 一致確認ができた場合、成功メッセージを表示
    print("✅ shapes & props match")

# ============================================================
# サニティチェックの実行
# ------------------------------------------------------------
# `train_dataloader` から最初のバッチを取得し、`sanity_check` 関数でデータとプロパティを比較します。
# これにより、データの形式が正しいことを確認します。
# ------------------------------------------------------------
first_batch = next(iter(train_dataloader))
sanity_check(first_batch, data_props)


In [ ]:
hist_cardinality = hist_max + 1      # 0‑start なら +1 でテーブルサイズ
data_props['historical_categorical_cardinalities'] = [hist_cardinality]
data_props['num_historical_categorical']            = 1     # 列は 1 本

# static も同じように自動で
static_min  = int(first_batch['static_feats_categorical'].min())
static_max  = int(first_batch['static_feats_categorical'].max())
static_card = static_max + 1
data_props['static_categorical_cardinalities'] = [static_card] * \
                                                 data_props['num_static_categorical']


In [ ]:
# ============================================================
# サニティチェックの実行
# ------------------------------------------------------------
# `train_dataloader` から最初のバッチを取得し、`sanity_check` 関数でデータとプロパティを比較します。
# これにより、データの形式が正しいことを確認します。
# ------------------------------------------------------------
first_batch = next(iter(train_dataloader))
sanity_check(first_batch, data_props)  # サニティチェックを実行

# 歴史的カテゴリ特徴量が指定したカーディナリティを超えていないかのチェック
first_batch = next(iter(train_dataloader))
assert first_batch['historical_ts_categorical'].max() < \
       data_props['historical_categorical_cardinalities'][0], "⚠ still overflow!"

# サニティチェックが問題なく終了した場合、トレーニング開始
print("✅ all good – start training")

In [ ]:
print(data_props)
first_batch = next(iter(train_dataloader))
sanity_check(first_batch, data_props)

In [ ]:
np.unique(g_score)
# または
np.unique(g_score.reshape(-1))


In [ ]:
print(static.columns.tolist())
for col in static.columns:
    print(f"{col}: {static[col].nunique()}種類")


In [ ]:
# ============================================================
# 量子損失（Quantile Loss）計算
# ------------------------------------------------------------
# 各時点、変数、分位点ごとに Quantile Loss を計算する関数
# モデルの出力（予測結果）とターゲット（実際の値）との間で損失を計算
# 損失の計算には Pinball Loss を使用し、欠損値を考慮したマスクを適用する
# ------------------------------------------------------------
def compute_quantile_loss_instance_wise(outputs: torch.Tensor,  # モデルの出力（B, T, F, Q）
                                        targets: torch.Tensor,  # 正解値（B, T, F）
                                        masks: torch.Tensor,    # 欠損データ用マスク（B, T, F）
                                        desired_quantiles: torch.Tensor) -> torch.Tensor:  # 分位点リスト（Q）
    # エラー（予測とターゲットの差）を計算
    errors = targets.unsqueeze(-1) - outputs

    # 欠損データに対してマスクを適用（無効なデータは損失の計算に含めない）
    for i in range(masks.shape[-1]):  # 各変数について
        for j in range(len(desired_quantiles)):  # 各分位点について
            errors[..., i, j] = errors[..., i, j] * masks[..., i]  # マスク適用

    # ピンボール損失（Quantile Loss）を計算
    losses_array = torch.max((desired_quantiles - 1) * errors, desired_quantiles * errors)
    return losses_array  # [B, T, F, Q]

# ============================================================
# 量子損失および Q-risk の計算
# ------------------------------------------------------------
# モデルの出力（predicted_quantiles）を使って、損失（q_loss）と分位点ごとのリスク（q_risk）を計算する関数
# 1) ピンボール損失を計算し、全体平均損失（q_loss）を算出
# 2) 各分位点ごとのリスク（q_risk）を算出
# ------------------------------------------------------------
def get_quantiles_loss_and_q_risk(outputs: torch.Tensor,  # 予測された分位点（B, T, F×Q）
                                  targets: torch.Tensor,  # 実際のターゲット（B, T, F）
                                  masks: torch.Tensor,    # 欠損データ用マスク（B, T, F）
                                  desired_quantiles: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    
    B, T, FQ = outputs.shape
    Q = desired_quantiles.numel()  # 分位点の数を自動取得
    F = FQ // Q  # 変数数を計算

    # 予測結果を [B, T, F, Q] に reshape
    outputs = outputs.view(B, T, F, Q)

    # 1) ピンボール損失の計算
    losses_array = compute_quantile_loss_instance_wise(
        outputs=outputs,
        targets=targets,
        masks=masks,
        desired_quantiles=desired_quantiles
    )

    # 2) 全体の平均損失（q_loss）
    q_loss = (
        losses_array.sum(dim=-1)  # 分位点に関する合計
        .sum(dim=-1)  # 変数に関する合計
        .mean(dim=-1)  # 時系列に関する平均
        .mean()  # バッチサイズに関する平均
    )

    # 3) 分位点ごとの Q-risk
    denom = targets.abs().sum() + 1e-8  # 0 除算防止
    q_risk = 2 * losses_array.sum(dim=(0, 1)) / denom  # [F, Q]
    q_risk = q_risk.sum(dim=0)  # [Q]

    return q_loss, q_risk, losses_array

# ============================================================
# バッチ処理と損失計算
# ------------------------------------------------------------
# バッチデータをモデルに通し、損失（q_loss）と Q-risk（q_risk）を計算する関数
# モデルの出力である predicted_quantiles とターゲット値を使って最終的な損失とリスクを計算
# ------------------------------------------------------------
def process_batch(batch: Dict[str, torch.Tensor],
                  model: nn.Module,
                  quantiles_tensor: torch.Tensor,
                  device: torch.device):

    # モデルが GPU を使う場合、バッチデータをデバイスに転送
    if device.type == "cuda":
        for k in list(batch.keys()):
            try:
                # Embedding層用にint型→longに変換
                if batch[k].dtype == torch.int32:
                    batch[k] = batch[k].to(torch.long)
                batch[k] = batch[k].to(device)
            except Exception as e:
                print(f"[ERROR] Failed to move '{k}' to device.")
                print(f"Type: {type(batch[k])}")
                print(f"Value: {batch[k]}")
                raise e

    # モデルの出力を取得（分位点の予測結果）
    batch_outputs = model(batch)  # batch_outputs['predicted_quantiles']  # [B, T, F×Q]
    
    # ターゲットデータとマスクをデバイスに転送
    labels = batch['target'].to(device)
    target_masks = batch['target_mask'].to(device)
    
    # モデルの予測分位点（[B, T, F×Q]）を取得
    predicted_quantiles = batch_outputs['predicted_quantiles']
    
    # 損失とリスクを計算
    q_loss, q_risk, _ = get_quantiles_loss_and_q_risk(outputs=predicted_quantiles,
                                                      targets=labels,
                                                      masks=target_masks,
                                                      desired_quantiles=quantiles_tensor)
    return q_loss, q_risk


In [ ]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(device)


In [ ]:
# ==============================
# ⏪ 時系列ステップ設定
# ==============================
# 過去データをどれだけ遡って学習に使うか（例：過去◯ヶ月）
historical_steps = past_months

# 未来データをどれだけ先まで予測するか（例：未来◯ヶ月）
future_steps = future_months

# ==============================
# 🔁 学習設定
# ==============================
# 学習の反復回数（エポック数）
num_epochs = 100

# ==============================
# 📉 移動平均ウィンドウ（損失スムージング用）
# ==============================
# QueueAggregator で使用する移動平均のウィンドウサイズ。
# 意味：直近 5 エポックの損失を移動平均でスムージングする。
# 用途：損失の短期的な変動をならし、学習の進行を安定的に観察するため。
ma_queue_size = 5
patience_limit =500
# ============================================================
# 訓練設定 (Training Configuration)
# ------------------------------------------------------------
# モデルのハイパーパラメータと訓練の設定を含む辞書です。
# 各パラメータは、学習率、バッチサイズ、モデル構造、ドロップアウト率などを指定します。
# ------------------------------------------------------------

configuration = {
    'optimization': {
        # バッチサイズ: 一回の訓練で使用するデータの数
        'batch_size': b_size,

        # 学習率: モデルのパラメータを更新する速さ
        'learning_rate': 1e-4,

        # 勾配クリッピング: 勾配のノルムが大きすぎると学習が不安定になるため、これを制限
        'max_grad_norm': 5,  # 勾配のノルム制限
    },

    'model': {
        # ドロップアウト率: 訓練中にランダムにノードを無効化し、過学習を防ぐ
        'dropout': 0.4,  # 過学習防止

        # 隠れ状態のサイズ: LSTM や Attention における隠れ層のノード数
        'state_size': 192,  # モデルの「幅」

        # 出力する分位点（ピンボール損失用）
        'output_quantiles': [0.25, 0.5, 0.75],  # ピンボール損失用の分位点

        # LSTM レイヤー数: モデルの深さ。層を増やすことで表現力が向上します
        'lstm_layers': 2,  # LSTMレイヤー数

        # Attention ヘッド数: マルチヘッドAttentionにおけるヘッド数
        'attention_heads': 2,  # 複数の視点で情報を処理
    },

    'task_type': 'regression',  # タスクタイプ（回帰問題として設定）

    # 予測対象の開始位置: 特に指定がなければ自動設定される
    'target_window_start': None,  # 予測対象の開始位置

    # データプロパティ情報
    'data_props': data_props  # モデルに与えるデータセットの情報
}


In [ ]:
# CUDAが利用可能ならGPUを、そうでなければCPUを使用
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
print(device)

In [ ]:
# ==============================
# ⚙️ TFTモデルの初期化
# ==============================
# configuration: モデルや学習のハイパーパラメータを含む設定辞書
tft_model = TFT_model.TemporalFusionTransformer(OmegaConf.create(configuration))

# GPUが1枚しかない場合は DataParallel を使用しない
# （複数GPUがある場合は下のコメントアウトを有効化）
# tft_model = nn.DataParallel(tft_model, device_ids=[0, 1])

# モデルを指定デバイス（GPU or CPU）に転送
tft_model.to(device)
print(device)

# ==============================
# 📦 データローダーの設定
# ==============================
train_dataloader = DataLoader(train_dataset, batch_size=b_size, shuffle=True, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=b_size, shuffle=True, num_workers=4)

# ==============================
# 🧠 最適化手法（Optimizer）の設定
# ==============================
opt = optim.Adam(
    filter(lambda p: p.requires_grad, list(tft_model.parameters())),
    lr=configuration['optimization']['learning_rate']
)

# ==============================
# 📉 損失値の移動平均（Moving Average）トラッカー
# ==============================
# QueueAggregator は、直近 max_size 個の損失値をキューで保持し、平均値を計算するクラス。
# 損失の変動を滑らかにし、安定した学習挙動をモニタリングできる。
loss_aggregator = QueueAggregator(max_size=ma_queue_size)

# ==============================
# 🔢 分位点テンソル（Quantile Tensor）の設定
# ==============================
# TFTは分位点予測（Quantile Forecasting）を行うモデル。
# configuration 内の設定値から使用する分位点をテンソル化。
quantiles_tensor = torch.tensor(configuration['model']['output_quantiles']).to(device)


In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

loss_arr = []
loss_arr_test = []
patience = 0
min_loss = 9999
best_model = tft_model
best_test_loss = float('inf')

for epoch in range(num_epochs):
    loss_e = 0
    loss_e_test = 0

    
    # ==============================
    #       Train Phase
    # ==============================
    tft_model.train()
    for data in train_dataloader:
        opt.zero_grad()
        
        loss, _ = process_batch(batch=data,
                                model=tft_model,
                                quantiles_tensor=quantiles_tensor,
                                device=device)
        
        loss_e += loss.item()
        loss.backward()
        
        if configuration['optimization']['max_grad_norm'] > 0:
            nn.utils.clip_grad_norm_(tft_model.parameters(), configuration['optimization']['max_grad_norm'])

        opt.step()
        loss_aggregator.append(loss.item())
        
    loss_arr.append(loss_e)

    # ==============================
    #       Early Stopping
    # ==============================
    if len(loss_arr) > 1:
        if min_loss > loss_arr[-1]:
            min_loss = loss_arr[-1]
            best_model = tft_model
            patience = 0
        else:
            patience += 1
            if patience > patience_limit:
                torch.save(best_model, "TFT-multi_earlystop.pt")
                np.savetxt("TFT-multi_train_loss.txt", loss_arr)
                np.savetxt("TFT-multi_test_loss.txt", loss_arr_test)
                print("🛑 Patience limit reached, exiting training.")
                break

    # ==============================
    #       Test Evaluation
    # ==============================
    tft_model.eval()
    with torch.no_grad():
        q_loss_eval, q_risk_eval = [], []
        for test_data in test_dataloader:
            batch_loss, batch_q_risk = process_batch(batch=test_data,
                                                     model=tft_model,
                                                     quantiles_tensor=quantiles_tensor,
                                                     device=device)
            loss_e_test += batch_loss.item()
            q_loss_eval.append(batch_loss)
            q_risk_eval.append(batch_q_risk)

        eval_loss = torch.stack(q_loss_eval).mean(axis=0)
        eval_q_risk = torch.stack(q_risk_eval, axis=0).mean(axis=0)
        loss_arr_test.append(loss_e_test)

    # ==============================
    #       Best Test Loss 保存
    # ==============================
    if loss_e_test < best_test_loss:
        best_test_loss = loss_e_test
        torch.save(tft_model, "TFT-multi_minmax.pt")
        np.savetxt("TFT-multi_train_loss.txt", loss_arr)
        np.savetxt("TFT-multi_test_loss.txt", loss_arr_test)
        print(f"✅ Saved new best model at epoch {epoch} with test loss {best_test_loss:.4f}")

    # ==============================
    #       定期保存
    # ==============================
    if epoch % 50 == 49:
        torch.save(tft_model, f"TFT-multi_36_{epoch}.pt")
        np.savetxt("TFT-multi_train_loss.txt", loss_arr)
        np.savetxt("TFT-multi_test_loss.txt", loss_arr_test)

    # ==============================
    #       ログ出力
    # ==============================
    print(f"Epoch: {epoch}, "
          f"Train Loss = {np.mean(loss_aggregator.get()):.5f}, "
          f"Test q_loss = {eval_loss:.5f}, " +
          ", ".join([f"q_risk_{q:.1f} = {risk:.5f}" for q, risk in zip(quantiles_tensor, eval_q_risk)]))


<h1>可視化と誤差計算</h1>


In [ ]:
# ============================================================
# 📦 モジュールの読み込みとパス設定
# ============================================================

import sys
import matplotlib.pyplot as plt
from matplotlib import font_manager as fm, rcParams

# フォントパスを登録
font_path = '/usr/share/fonts/opentype/ipafont-gothic/ipagp.ttf'
fm.fontManager.addfont(font_path)
rcParams['font.family'] = 'IPAPGothic'  # 名前は固定
rcParams['axes.unicode_minus'] = False


sys.path.append("/home/nishimura/seminar2025/M1/Nishimura/TFT/TFT-multi_csv/")

# visualization モジュールを再読み込み（修正を即時反映）
import visualization as vis
importlib.reload(vis)

# 実際にインポートされたファイルのパスを確認（デバッグ用途）
print(vis.__file__)

In [ ]:
# ============================================================
# ⚙️ Temporal Fusion Transformer モデルのロード
# ============================================================

# OmegaConfを使用して設定をTFTモデルに渡す
# ※ configuration は事前に定義された辞書やYAML設定ファイルを読み込んだもの
tft_model = TFT_model.TemporalFusionTransformer(OmegaConf.create(configuration))

# ------------------------------------------------------------
# 🔧 PyTorch 2.6以降では torch.load() の既定が weights_only=True
#    → モデル定義クラスがロード時に見つからないと UnpicklingError が発生する
#    → そのため、明示的に weights_only=False を指定してフルロードする
# ------------------------------------------------------------
tft_model = torch.load("TFT-multi_minmax.pt",weights_only=False)

# モデルを GPU または CPU に転送（環境に応じて device が定義されている前提）
tft_model.to(device)

# 評価モードに切り替え（ドロップアウトやバッチ正規化を無効化）
tft_model.eval()

# ------------------------------------------------------------
# ✅ これで学習済みTFTモデルがロード完了。
#    以降は推論や可視化（vis モジュール）で利用可能。
# ------------------------------------------------------------

In [ ]:
# ============================================================
# 🎨 グラフ表示設定（Matplotlibの描画設定）
# ============================================================
from matplotlib import rcParams

rcParams.update({
    'figure.autolayout': True,  # 図が自動で見やすくレイアウトされるように
    'figure.figsize': [10, 5],  # デフォルトの図サイズ
    'font.size': 17             # フォントサイズ
})

# ============================================================
# 🔍 process_test_batch()
# ------------------------------------------------------------
# テストデータの1バッチをモデルに入力して推論を行い、
# 分位点予測・損失（Quantile Loss, Q-Risk）を計算する関数。
#
# 【主な処理】
# 1. GPU対応：入力データをdeviceに転送
# 2. モデルによる予測を実行
# 3. 出力（predicted_quantiles）と正解（labels）を取得
# 4. 分位点損失とQ-Riskを計算
# 5. 結果をCPUに戻して返す（NumPy形式）
#
# 【想定用途】
# - 学習後のモデル検証
# - サンプル可視化（1バッチ分の予測 vs 正解）
#
# 【引数】
# batch            : DataLoaderから取得した1バッチ（辞書形式）
# model            : Temporal Fusion Transformer モデル
# quantiles_tensor : 予測で使用する分位点（例: torch.tensor([0.1, 0.5, 0.9])）
# device           : 使用デバイス（CPU or GPU）
#
# 【返り値】
# q_loss           : Quantile Loss
# q_risk           : Q-Risk
# predicted_quantiles : 予測値（NumPy形式）
# labels              : 正解値（NumPy形式）
# ============================================================

def process_test_batch(batch: Dict[str, torch.tensor],
                       model: nn.Module,
                       quantiles_tensor: torch.tensor,
                       device: torch.device):
    # ------------------------------------------------------------
    # 1. 各テンソルをデバイス（GPU/CPU）に転送
    # ------------------------------------------------------------
    if device.type == "cuda":
        for k in list(batch.keys()):
            batch[k] = batch[k].to(device)

    # ------------------------------------------------------------
    # 2. モデルによる推論
    # ------------------------------------------------------------
    # モデルの出力は辞書形式（例: {"predicted_quantiles": tensor(...)})
    batch_outputs = model(batch)

    # ------------------------------------------------------------
    # 3. 正解データ（target）とマスクを取得
    # ------------------------------------------------------------
    labels = batch['target']        # [batch, future_months, num_feat]
    target_masks = batch['target_mask']  # 予測対象を示す0/1マスク

    print(labels)  # （デバッグ用途）正解ラベルを表示

    # ------------------------------------------------------------
    # 4. 予測分位点を取得
    # ------------------------------------------------------------
    # モデルの出力テンソルには将来値の各分位点が格納されている
    predicted_quantiles = batch_outputs['predicted_quantiles']
    print(predicted_quantiles)  # （デバッグ用途）分位点予測を表示

    # ------------------------------------------------------------
    # 5. Quantile Loss と Q-Risk の計算
    # ------------------------------------------------------------
    q_loss, q_risk, _ = get_quantiles_loss_and_q_risk(
        outputs=predicted_quantiles,
        targets=labels,
        masks=target_masks,
        desired_quantiles=quantiles_tensor
    )

    # ------------------------------------------------------------
    # 6. GPU上のテンソルをCPUに戻してNumPy配列に変換
    # ------------------------------------------------------------
    # そのまま NumPy に変換するとエラーになるため .cpu() を明示的に呼ぶ
    return (
        q_loss,
        q_risk,
        predicted_quantiles.cpu().numpy(),
        labels.cpu().numpy()
    )

# ============================================================
# ✅ 使用例（1バッチをテスト実行）
# ------------------------------------------------------------
# q_loss, q_risk, preds, labels = process_test_batch(
#     next(iter(test_dataloader)),
#     tft_model,
#     quantiles_tensor=torch.tensor([0.1, 0.5, 0.9]),
#     device=device
# )
# ============================================================


In [ ]:
# ============================================================
# 🧭 full_cols（全特徴量の列名リスト）と target（予測対象）の定義
# ------------------------------------------------------------
# full_cols :
#   モデルの入力・出力に使用する全ての変数（特徴量）の名称をリスト化。
#   この順番が学習データの列順に対応している。
#
# target :
#   モデルが予測する目的変数の名称。
#   今回は「エリア需要」「純供給」「価格(円/kWh)」の3つ。
# ============================================================

lab = [
    "エリア需要","原子力","火力","水力","地熱","バイオマス",
    "太陽光発電実績","太陽光出力制御量","風力発電実績","風力出力制御量",
    "揚水","連系線","蓄電池","降水量(mm)","風速(m/s)","気温(℃)","価格(円/kWh)"
]

# 🔹 「純供給」を追加したフルカラム（全変数リスト）
full_cols = lab + ["純供給"]

# 🎯 予測対象の変数（ターゲット）
target = ["エリア需要", "純供給", "価格(円/kWh)"]

# ============================================================
# 🔍 各ターゲット変数が full_cols 内のどの列にあるかを特定
# ------------------------------------------------------------
# 例：
#   full_cols = [..., "エリア需要", ..., "価格(円/kWh)", ..., "純供給"]
#   target = ["エリア需要", "純供給", "価格(円/kWh)"]
#   → idxs = [0, 17, 16]
# ============================================================
idxs = [full_cols.index(c) for c in target]

# 結果表示
print("🎯 Target column indexes in full_cols:", idxs)


In [ ]:
# ============================================================
# 🎯 ターゲット列（目的変数）のインデックス取得
# ------------------------------------------------------------
# full_cols : モデルの入力・出力に使っている全特徴量の列名リスト
# target    : 予測対象の列名リスト（例：['エリア需要', '価格', '純供給']）
#
# 目的：
# 各ターゲット変数（需要・価格・供給など）が full_cols 内で何番目に
# 位置しているかを調べ、その列番号（index）を取得する。
# ============================================================

# full_cols の中で target 各要素がどの列に対応するかをリストで取得
idxs = [full_cols.index(c) for c in target]   # 例：[0, 16, 17]

# meas はどのターゲットを使うかを指定する変数
# 例：
#   meas=0 → エリア需要
#   meas=1 → 価格
#   meas=2 → 純供給
meas = 1   # 今回は「価格」を指定

# 対象列の index を取得
col_i = idxs[meas]   # meas=1 → idxs[1] = 16

# 確認出力
print("Target variable indexes:", idxs)
print(f"Selected variable index (meas={meas}):", col_i)


In [ ]:
# ============================================================
# 🔁 inverse_single_variable()
# ------------------------------------------------------------
# 正規化済みの配列から「1変数のみ」を元のスケールに戻す関数。
#
# 【背景】
# StandardScaler や MinMaxScaler は複数列をまとめて fit しているため、
# 1列だけを逆変換しようとすると shape が合わずエラーになる。
# そこで、一時的な「ダミー配列」を使って 1変数分だけ逆変換を行う。
#
# 【引数】
# arr      : ndarray
#     - 逆変換したい変数の正規化済み配列（形状: [n_samples] または [n_samples, 1]）
# col_idx  : int
#     - full_cols（スケーラーで fit した全カラム）の中で、この変数が何番目かを指定。
#       例：full_cols = ['売上', '価格', '在庫'] のとき、
#            価格を逆変換したい場合 col_idx = 1
#
# 【前提変数（外部で定義済み）】
# scaler   : fit済みの StandardScaler / MinMaxScaler など
# full_cols: スケーラーで fit した全列名リスト
#
# 【戻り値】
# ndarray : arr と同じ形状の、逆変換後（元スケール）の配列
#
# 【処理の流れ】
# 1️⃣ full_cols の次元に合わせたゼロ行列を作成
# 2️⃣ 対象列（col_idx）に arr の値を代入
# 3️⃣ scaler.inverse_transform() を実行
# 4️⃣ 対象列だけ取り出して返す
#
# 【使用例】
# y_pred_inv = inverse_single_variable(y_pred_norm, col_idx=1)
# ============================================================


def inverse_single_variable(arr, col_idx):
    """
    1変数だけスケーラーの逆変換を行う。
    arr: 正規化済みの1変数の配列
    col_idx: その変数が full_cols 内での列インデックス
    """
    # ------------------------------------------------------------
    # full_cols（スケーラーが fit した全列）と同じ形状のダミー配列を作る
    # ------------------------------------------------------------
    dummy = np.zeros((arr.shape[0], len(full_cols)))

    # 対象変数（col_idx列）に値を代入
    dummy[:, col_idx] = arr

    # ------------------------------------------------------------
    # 逆変換を実行（他の列は0のままなので影響しない）
    # ------------------------------------------------------------
    inv = minmaxscaler.inverse_transform(dummy)

    # 対象変数のみを取り出して返す
    return inv[:, col_idx]


In [ ]:
import matplotlib.font_manager
for f in matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf'):
    if 'ipa' in f.lower():
        print(f)


In [ ]:
# ===========================================
# Temporal Fusion Transformer (TFT) 予測可視化＆評価
#   - テストデータに対して予測を実行
#   - 各出力変数（需要・供給・価格）を逆正規化して描画
#   - 分位点（P10–P90）区間を表示
#   - 各変数の MAE（平均絶対誤差）を算出・出力
# ===========================================

# 出力変数の名前（可視化・評価用）
output_names = ["需要", "純供給", "価格"]

# 可視化・評価結果を格納するリスト
inv_hists = []  # 逆正規化した履歴系列
inv_trues = []  # 逆正規化した真値系列
inv_preds = []  # 逆正規化した予測系列
all_mae = [[] for _ in range(len(output_names))]  # 各出力変数ごとの MAE 保存用

# ===========================================
# 1. テストデータでモデルを評価
# ===========================================
with torch.no_grad():
    for data in test_dataloader:
        # --- モデル出力を取得 ---
        batch_loss, batch_q_risk, pred, true = process_test_batch(
            batch=data,
            model=tft_model,
            quantiles_tensor=quantiles_tensor,
            device=device
        )

        B, T, FQ = pred.shape  # (バッチ, 時間, 変数×分位点)
        Q = 3                  # 分位点の数（例：P10, P50, P90）
        F = FQ // Q            # 出力変数の数（需要・供給・価格）

        # ===========================================
        # 2. 各系列ごとにプロット（最大3系列）
        # ===========================================
        for idx in range(min(B, 3)):  # 最初の3系列だけ描画
            fig, axes = plt.subplots(F, 1, figsize=(12, 4 * F))
            if F == 1:
                axes = [axes]  # 変数が1つのときに対応

            for meas in range(F):
                col_i = idxs[meas]  # full_cols の中での列インデックス
                pred_batch = pred[..., 3 * meas: 3 * (meas + 1)]  # 該当変数の分位点部分
                true_batch = true[..., meas]

                # --- 逆正規化（スケーラーを使って実スケールに戻す） ---
                inv_hist = inverse_single_variable(
                    data['historical_ts_numeric'][idx, :, -4 + meas].cpu().numpy(),
                    col_i
                )
                inv_true = inverse_single_variable(true_batch[idx], col_i)
                inv_pred = np.stack([
                    inverse_single_variable(pred_batch[idx, :, q], col_i)
                    for q in range(3)
                ], axis=-1)

                # --- P50（中央値）予測 ---
                pred_median = inv_pred[:, 1]

                # --- MAE（平均絶対誤差）を計算 ---
                mae = np.mean(np.abs(pred_median - inv_true))
                all_mae[meas].append(mae)

                # ===========================================
                # 3. 可視化（履歴＋真値＋予測＋予測区間）
                # ===========================================
                ax = axes[meas]
                ax.plot(range(len(inv_hist)), inv_hist,
                        label='History', color='black', linewidth=2)
                ax.plot(range(len(inv_hist), len(inv_hist) + len(inv_true)), inv_true,
                        label='True', color='green', linewidth=2)
                ax.plot(range(len(inv_hist), len(inv_hist) + len(inv_true)), pred_median,
                        label='Pred (P50)', color='blue', linewidth=2)

                # --- 予測区間 (P10–P90) ---
                ax.fill_between(
                    range(len(inv_hist), len(inv_hist) + len(inv_true)),
                    inv_pred[:, 0], inv_pred[:, 2],
                    color='gray', alpha=0.3, label='P10–P90'
                )

                # --- タイトルに MAE を表示 ---
                ax.set_title(f'Series {idx}, {output_names[meas]} (MAE={mae:.3f})')
                ax.legend()
                ax.grid(True)

            plt.tight_layout()
            plt.show()

# ===========================================
# 4. 変数ごとの平均MAEを出力
# ===========================================
target_names = ["需要", "供給", "価格"]
print("=== 平均 MAE ===")
for i in range(len(target_names)):
    mae_mean = np.mean(all_mae[i]) if len(all_mae[i]) > 0 else float('nan')
    mae_median = np.median(all_mae[i]) if len(all_mae[i]) > 0 else float('nan')
    print(f"{target_names[i]}: 平均={mae_mean:.3f}  中央値={mae_median:.3f}")


In [ ]:
# ============================================================
# 🌏 地域別 × 予測変数別 MAE（Mean Absolute Error）算出
# ------------------------------------------------------------
# ・テストデータに含まれる各系列を順に推論
# ・地域（中国・四国）ごとに MAE を集計
# ・逆正規化（inverse transform）後の元スケールで評価
# ============================================================

# DataLoader 内の系列IDに対応（例：中国_01、四国_05 など）
sequence_ids_test = sequence_ids[test_set]

# 分析対象地域と予測対象変数
regions = ["中国", "四国"]
target_names = ["需要", "供給", "価格"]
num_features_predicted = len(target_names)  # 予測対象変数数 (=3)

# 元データ内での対象変数の列インデックス（例：需要=0, 供給=17, 価格=16）
idxs = [0, 17, 16]

# 地域 × 変数 ごとの MAE 値を保存する辞書
region_mae = {r: [ [] for _ in range(num_features_predicted) ] for r in regions}

# DataLoader の各サンプルを追跡するためのカウンタ
counter = 0

# ------------------------------------------------------------
# 推論処理ループ
# ------------------------------------------------------------
with torch.no_grad():
    for data in test_dataloader:
        # モデルで推論を実行
        _, _, pred, true = process_test_batch(
            batch=data,
            model=tft_model,
            quantiles_tensor=quantiles_tensor,
            device=device
        )

        # pred: [B, T, F×Q], true: [B, T, F]
        B, T, FQ = pred.shape
        Q = 3  # 分位点数（P10, P50, P90）
        F = FQ // Q

        # Tensor → NumPy に変換
        if isinstance(pred, torch.Tensor):
            pred = pred.detach().cpu().numpy()
        if isinstance(true, torch.Tensor):
            true = true.detach().cpu().numpy()

        # --------------------------------------------------------
        # 各系列（b）ごとに地域を判定してMAEを計算
        # --------------------------------------------------------
        for b in range(B):
            seq_id = sequence_ids_test[counter]  # 系列のIDを取得（例：中国_03）
            counter += 1

            # 地域判定
            if "中国" in seq_id:
                region = "中国"
            elif "四国" in seq_id:
                region = "四国"
            else:
                continue  # 対象外はスキップ

            # 各出力変数（需要・供給・価格）について評価
            for m in range(num_features_predicted):
                col_i = idxs[m]  # スケーラー逆変換用の列インデックス

                # 中央予測 (P50)
                pred_median = pred[b, :, m*Q + 1]
                true_vals   = true[b, :, m]

                # 逆変換（正規化 → 元スケールへ）
                inv_pred = inverse_single_variable(pred_median, col_i)
                inv_true = inverse_single_variable(true_vals, col_i)

                # 平均絶対誤差 (MAE)
                mae = np.mean(np.abs(inv_pred - inv_true))
                region_mae[region][m].append(mae)

# ------------------------------------------------------------
# 結果出力
# ------------------------------------------------------------
print("=== 🌏 地域別 MAE（逆変換後, 元スケール） ===")
for region in regions:
    for m, name in enumerate(target_names):
        vals = np.array(region_mae[region][m])
        if len(vals):
            print(f"{region} {name}: 平均MAE = {vals.mean():.3f}  中央MAE = {np.median(vals):.3f}  (n = {len(vals)})")
        else:
            print(f"{region} {name}: データなし")
